In [1]:
import praw
import pandas as pd
import datetime
import concurrent.futures

reddit = praw.Reddit(client_id='_37znztIqSaRtA8ZWOCRXg', client_secret="0wz5rUMlMjJBSCwUuXhuDt_64OOCyA",
                     user_agent='CIS509')

subreddit = reddit.subreddit('GrubHub')

def process_submission(submission):
    try:
        submission.comments.replace_more(limit=0)
        comments_data = []
        for comment in submission.comments.list():
            timestamp = datetime.datetime.fromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S')
            comments_data.append([comment.id,
                                 comment.author,
                                 timestamp,
                                 comment.parent_id,
                                 submission.id,
                                 submission.title,
                                 comment.body,
                                 comment.permalink])
        return comments_data
    except Exception as e:
        print(f"Error processing submission with id {submission.id}: {e}")
        return []

comments_data = []
count = 0
with concurrent.futures.ThreadPoolExecutor() as executor:
    submission_futures = [executor.submit(process_submission, submission) for submission in subreddit.new(limit=1000)]
    for submission_future in concurrent.futures.as_completed(submission_futures):
        comments_data += submission_future.result()
        count += len(submission_future.result())
        print(f"Processed {count} comments.")
        if count >= 2000:
            break

comments_df = pd.DataFrame(comments_data, columns=['Comment ID', 'Author', 'Timestamp',
                                                    'Comment is a reply to', 'Comment Thread ID',
                                                    'Thread Title', 'Comment Body', 'Comment Permalink'])

print(comments_df)


Version 7.6.1 of praw is outdated. Version 7.7.0 was released Saturday February 25, 2023.


Processed 0 comments.
Processed 5 comments.
Processed 8 comments.
Processed 11 comments.
Processed 13 comments.
Processed 17 comments.
Processed 31 comments.
Processed 36 comments.
Processed 41 comments.
Processed 43 comments.
Processed 44 comments.
Processed 51 comments.
Processed 52 comments.
Processed 60 comments.
Processed 66 comments.
Processed 69 comments.
Processed 82 comments.
Processed 94 comments.
Processed 100 comments.
Processed 106 comments.
Processed 111 comments.
Processed 123 comments.
Processed 134 comments.
Processed 147 comments.
Processed 151 comments.
Processed 178 comments.
Processed 187 comments.
Processed 210 comments.
Processed 214 comments.
Processed 217 comments.
Processed 220 comments.
Processed 225 comments.
Processed 229 comments.
Processed 236 comments.
Processed 256 comments.
Processed 260 comments.
Processed 276 comments.
Processed 280 comments.
Processed 281 comments.
Processed 284 comments.
Processed 285 comments.
Processed 290 comments.
Processed 298

In [2]:
comments_df.to_csv('Grubhub_comments_21_03.csv',index=False)

In [3]:
!pip install textblob

In [4]:
!pip install nltk

In [5]:
!pip install pandas

In [6]:
import pandas as pd
data = pd.read_csv('Grubhub_comments_21_03.csv')
data.head(2)

,Comment ID,Author,Timestamp,Comment is a reply to,Comment Thread ID,Thread Title,Comment Body,Comment Permalink
0,iil40i1,BetLongjumping5132,2022-08-01 18:22:40,t3_wdvln6,wdvln6,I’ve been banned from ordering anything becaus...,this happens to every customer and driver even...,/r/grubhub/comments/wdvln6/ive_been_banned_fro...
1,il7ghcl,Patient-Gain5847,2022-08-21 09:08:29,t3_wdvln6,wdvln6,I’ve been banned from ordering anything becaus...,I’m going through this right now. Every order ...,/r/grubhub/comments/wdvln6/ive_been_banned_fro...


In [7]:
#Parse csv's to only extract column called 'message'
messages = data['Comment Body']

In [8]:
from textblob import TextBlob
import re
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
#Text pre-processing functions
def text_processing(message):
    
    #Generating the list of words in the message (hastags and other punctuations removed) and convert to lowercase
    def form_sentence(message):
        message = message.lower() #Make messages lowercase 
        message_blob = TextBlob(message.lower()) #Convert to 'textblob' which provides a simple API for NLP tasks
        return ' '.join(message_blob.words)
    new_message = form_sentence(message)
    
    #Removing stopwords and words with unusual symbols
    def no_user_alpha(message):
        message_list = [item for item in message.split()] 
        clean_words = [word for word in message_list if re.match(r'[^\W\d]*$', word)] #remove punctuation and strange characters
        clean_sentence = ' '.join(clean_words) 
        clean_mess = [stopword for stopword in clean_sentence.split() if stopword not in stopwords.words('english')] #remove stopwords
        return clean_mess
    no_punc_message = no_user_alpha(new_message)
    
    #Normalizing the words in messages 
    def normalization(message_list):
        lem = WordNetLemmatizer()
        normalized_message = []
        for word in message_list:
            normalized_text = lem.lemmatize(word,'v') #lemmatize words
            normalized_message.append(normalized_text)
        return normalized_message
    return normalization(no_punc_message)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sirik\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sirik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sirik\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\sirik\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [9]:
#Print to console and write to file
f = open('processed_text_grubhub.csv','w', encoding='utf8')
for message in messages: 
    message = text_processing(message)
    for term in message:
        f.write(term+" ")
    f.write("\n")

In [10]:
f.close()

In [11]:
!pip install spacy

In [12]:
!python -m spacy download en_core_web_lg

[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [13]:
!pip install pydantic --upgrade

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\sirik\\anaconda3\\Lib\\site-packages\\~-dantic\\annotated_types.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.




  Using cached pydantic-1.10.6-cp39-cp39-win_amd64.whl (2.2 MB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.8.2
    Uninstalling pydantic-1.8.2:
      Successfully uninstalled pydantic-1.8.2


In [14]:
!pip install blis

In [15]:
!pip install thinc

  Using cached pydantic-1.8.2-cp39-cp39-win_amd64.whl (1.9 MB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.6
    Uninstalling pydantic-1.10.6:
      Successfully uninstalled pydantic-1.10.6


In [16]:
!pip install typing_extensions>=3.7.4

In [17]:
import spacy
nlp = spacy.load("en_core_web_lg")
docs = list(nlp.pipe(data['Comment Body']))

In [18]:
import operator
np_dict = {}
for doc in docs:
    for np in doc.noun_chunks:
        np_dict[np.text]=np_dict.get(np.text,0)+1
sortedDict = sorted(np_dict.items(),key=operator.itemgetter(1),reverse=True)
print(sortedDict)

[('I', 2806), ('it', 1589), ('you', 1397), ('they', 991), ('that', 644), ('me', 480), ('them', 347), ('we', 303), ('this', 294), ('It', 290), ('They', 221), ('You', 212), ('what', 211), ('the driver', 210), ('the order', 204), ('the restaurant', 196), ('Grubhub', 174), ('who', 154), ('i', 152), ('GH', 148), ('us', 126), ('That', 121), ('We', 118), ('he', 117), ('food', 110), ('people', 110), ('drivers', 109), ('GrubHub', 103), ('which', 98), ('a driver', 94), ('the food', 91), ('something', 90), ('grubhub', 89), ('This', 88), ('the app', 83), ('someone', 82), ('the customer', 81), ('anything', 75), ('your order', 75), ('all', 73), ('the tip', 72), ('she', 68), ('orders', 67), ('delivery', 65), ('your food', 63), ('customers', 59), ('an order', 55), ('time', 52), ('a refund', 51), ('the time', 49), ('him', 49), ('my account', 48), ('nothing', 47), ('a lot', 47), ('money', 47), ('my food', 46), ('my order', 44), ('the drivers', 44), ('anyone', 43), ('What', 39), ('everything', 37), ('a c

In [19]:
entity_dict = {}
for doc in docs:
    for entity in doc.ents:
        entity_dict[entity.text]=entity_dict.get(entity.text,0)+1
sortedDict = sorted(entity_dict.items(),key=operator.itemgetter(1),reverse=True)
print(sortedDict)

[('Grubhub', 191), ('GrubHub', 166), ('GH', 106), ('2', 77), ('one', 69), ('first', 68), ('3', 60), ('two', 56), ('10', 53), ('1', 43), ('Amazon', 42), ('5', 38), ('Doordash', 30), ('DoorDash', 23), ('second', 21), ('third', 20), ('three', 19), ('PayPal', 18), ('20', 18), ('today', 16), ('15', 16), ('20%', 15), ('4', 15), ('6', 15), ('Walmart', 14), ('100', 13), ('One', 13), ('12', 12), ('years', 11), ('yesterday', 11), ('monthly', 11), ('an hour', 11), ('18', 11), ('100%', 10), ('25%', 10), ('last week', 9), ('UE', 9), ('5 miles', 9), ('half', 9), ('https://grubhub.com/help/contact-us', 8), ('tonight', 8), ('Ubereats', 8), ('ETA', 8), ('🤣', 8), ('BBB', 7), ('zero', 7), ('First', 7), ('hourly', 7), ('Reddit', 7), ('a mile', 7), ('9', 7), ('Spark', 7), ('eta', 7), ('20 miles', 7), ('gh+', 6), ('3 miles', 6), ('2nd', 6), ('3rd', 6), ("McDonald's", 6), ('30 minutes', 6), ('daily', 6), ('grubhub', 6), ('30', 6), ('Yesterday', 5), ('a month', 5), ('NYC', 5), ('last night', 5), ('NY', 5), ('

In [20]:
# To get SpaCy tag list :https://machinelearningknowledge.ai/tutorial-on-spacy-part-of-speech-pos-tagging/
from collections import Counter
verbs = []
for doc in docs:
    for token in doc: 
        if (not token.is_stop and not token.is_punct and token.pos_=="VERB"):
            verbs.append(token.lemma_)
verb_freq = Counter(verbs)
common_verbs = verb_freq.most_common(10)
print(common_verbs)

[('pay', 299), ('get', 291), ('deliver', 276), ('go', 241), ('know', 241), ('order', 238), ('happen', 219), ('say', 204), ('try', 171), ('tell', 169)]
